In [1]:
# ! pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

In [2]:
import os
import time
import copy
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchsummary import summary

from datetime import timedelta

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, \
precision_recall_fscore_support

from ipynb.fs.defs.research import to_tensor, normalise, reshape_tsr, CryptoImageDataset, \
ModelSaver, EarlyStoppingValidator, ModelTrainerTester, plot_learning_curves, plot_conf_matx,\
CNNGRU, CNN, GRU, ExecutionHandler

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
RANDOM_SEED = 11919925
BATCH_SIZE = 256
LR = 1e-3
TIMEPERIOD = 33
NUM_LOOPS = 30
DEVICE = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

TRAIN_CUTOFF = pd.to_datetime("2021-07-01 00:00:00+00:00")
VAL_CUTOFF = pd.to_datetime("2023-01-01 00:00:00+00:00")

In [5]:
# btc = pd.read_feather('./datasets/BTC_USDT-1h-new.feather')
# eth = pd.read_feather('./datasets/ETH_USDT-1h-new.feather')
# bnb = pd.read_feather('./datasets/BNB_USDT-1h-new.feather')

btc = pd.read_feather('./datasets/BTC_USDT-1h-test.feather')
eth = pd.read_feather('./datasets/ETH_USDT-1h-test.feather')
bnb = pd.read_feather('./datasets/BNB_USDT-1h-test.feather')

In [6]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

# Reporting statistics

## Training and Testing for the BTC dataset

In [7]:
BTC_CNNGRU_allstats = {}
BTC_CNN_allstats = {}
BTC_GRU_allstats = {}

In [8]:
# 1.1. Training and testing for BTC dataset
for i in range(NUM_LOOPS):
    print(f"ITERATION {i}:")
    
    # ==============================================================================
    # BTC - CNNGRU
    btc_exec_CNNGRU = ExecutionHandler(dataset=btc, model=CNNGRU,
                                       model_id=f"BTC-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(btc)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {btc_exec_CNNGRU.model.__class__.__name__}-{btc_exec_CNNGRU.model.model_id}")
    btc_exec_CNNGRU.run()
    # Append results
    model_name1 = next(iter(btc_exec_CNNGRU.statistics))
    BTC_CNNGRU_allstats[model_name1] = btc_exec_CNNGRU.statistics[model_name1]
    
    print(f"------------------------------------------------------------------")
    
    # ==============================================================================
    # BTC - CNN
    btc_exec_CNN = ExecutionHandler(dataset=btc, model=CNN,
                                       model_id=f"BTC-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(btc)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {btc_exec_CNN.model.__class__.__name__}-{btc_exec_CNN.model.model_id}")
    btc_exec_CNN.run()
    # Append results
    model_name2 = next(iter(btc_exec_CNN.statistics))
    BTC_CNN_allstats[model_name2] = btc_exec_CNN.statistics[model_name2]
    
    print(f"------------------------------------------------------------------")
    
    # ==============================================================================
    # BTC - GRU
    btc_exec_GRU = ExecutionHandler(dataset=btc, model=GRU,
                                       model_id=f"BTC-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(btc)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {btc_exec_GRU.model.__class__.__name__}-{btc_exec_GRU.model.model_id}")
    btc_exec_GRU.run()
    # Append results
    model_name3 = next(iter(btc_exec_GRU.statistics))
    BTC_GRU_allstats[model_name3] = btc_exec_GRU.statistics[model_name3]
    print(f"==============================================================================")

ITERATION 0:
Model: CNNGRU-BTC-ITER0
EPOCH [1/500],	 Train Loss: 0.85, Val Loss: 1.31, Train f1 (macro): 0.56, Val f1 (macro): 0.15, Elapsed time: 0m 21s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.53, Train f1 (macro): 0.91, Val f1 (macro): 0.42, Elapsed time: 3m 30s
EPOCH [20/500],	 Train Loss: 0.10, Val Loss: 0.47, Train f1 (macro): 0.96, Val f1 (macro): 0.43, Elapsed time: 7m 1s
EPOCH [30/500],	 Train Loss: 0.05, Val Loss: 0.37, Train f1 (macro): 0.98, Val f1 (macro): 0.45, Elapsed time: 10m 32s
Training completed in 10m 53s for CNNGRU.
Total epochs trained: 31
Best epoch: 20
Best val loss: 0.3221885576271094
Best val f1 (macro): 0.45821653230278675
------------------------------------------------------------------
Model: CNN-BTC-ITER0
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.79, Train f1 (macro): 0.68, Val f1 (macro): 0.35, Elapsed time: 0m 17s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.63, Train f1 (macro): 0.92, Val f1 (macro): 0.42, Elapsed time: 2m 49s
Training com

------------------------------------------------------------------
Model: CNN-BTC-ITER4
EPOCH [1/500],	 Train Loss: 0.64, Val Loss: 0.87, Train f1 (macro): 0.70, Val f1 (macro): 0.34, Elapsed time: 0m 17s
EPOCH [10/500],	 Train Loss: 0.20, Val Loss: 0.56, Train f1 (macro): 0.93, Val f1 (macro): 0.44, Elapsed time: 2m 50s
EPOCH [20/500],	 Train Loss: 0.17, Val Loss: 0.50, Train f1 (macro): 0.94, Val f1 (macro): 0.45, Elapsed time: 5m 45s
Training completed in 6m 19s for CNN.
Total epochs trained: 22
Best epoch: 11
Best val loss: 0.4754957906328715
Best val f1 (macro): 0.44805212649960674
------------------------------------------------------------------
Model: GRU-BTC-ITER4
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 1.28, Train f1 (macro): 0.69, Val f1 (macro): 0.29, Elapsed time: 0m 20s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.47, Train f1 (macro): 0.93, Val f1 (macro): 0.45, Elapsed time: 3m 26s
Training completed in 5m 50s for GRU.
Total epochs trained: 17
Best epoch: 6
Best v

EPOCH [1/500],	 Train Loss: 0.84, Val Loss: 1.16, Train f1 (macro): 0.58, Val f1 (macro): 0.18, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.21, Val Loss: 0.49, Train f1 (macro): 0.93, Val f1 (macro): 0.42, Elapsed time: 3m 47s
EPOCH [20/500],	 Train Loss: 0.10, Val Loss: 0.40, Train f1 (macro): 0.97, Val f1 (macro): 0.45, Elapsed time: 7m 35s
Training completed in 9m 6s for CNNGRU.
Total epochs trained: 24
Best epoch: 13
Best val loss: 0.3654449094946568
Best val f1 (macro): 0.4484108221764152
------------------------------------------------------------------
Model: CNN-BTC-ITER9
EPOCH [1/500],	 Train Loss: 0.68, Val Loss: 1.21, Train f1 (macro): 0.67, Val f1 (macro): 0.28, Elapsed time: 0m 18s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.52, Train f1 (macro): 0.92, Val f1 (macro): 0.44, Elapsed time: 3m 6s
EPOCH [20/500],	 Train Loss: 0.13, Val Loss: 0.45, Train f1 (macro): 0.96, Val f1 (macro): 0.45, Elapsed time: 6m 9s
Training completed in 7m 17s for CNN.
Total epochs tra

------------------------------------------------------------------
Model: CNN-BTC-ITER13
EPOCH [1/500],	 Train Loss: 0.68, Val Loss: 0.85, Train f1 (macro): 0.67, Val f1 (macro): 0.34, Elapsed time: 0m 17s
EPOCH [10/500],	 Train Loss: 0.21, Val Loss: 0.66, Train f1 (macro): 0.93, Val f1 (macro): 0.42, Elapsed time: 2m 44s
EPOCH [20/500],	 Train Loss: 0.18, Val Loss: 0.56, Train f1 (macro): 0.94, Val f1 (macro): 0.44, Elapsed time: 5m 26s
Training completed in 7m 37s for CNN.
Total epochs trained: 28
Best epoch: 17
Best val loss: 0.50632405682252
Best val f1 (macro): 0.4394056904605988
------------------------------------------------------------------
Model: GRU-BTC-ITER13
EPOCH [1/500],	 Train Loss: 0.68, Val Loss: 0.66, Train f1 (macro): 0.68, Val f1 (macro): 0.39, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.58, Train f1 (macro): 0.93, Val f1 (macro): 0.44, Elapsed time: 3m 32s
EPOCH [20/500],	 Train Loss: 0.23, Val Loss: 0.54, Train f1 (macro): 0.94, Val f1 (m

Training completed in 3m 50s for CNN.
Total epochs trained: 14
Best epoch: 3
Best val loss: 0.41925190675717133
Best val f1 (macro): 0.44622327048677546
------------------------------------------------------------------
Model: GRU-BTC-ITER17
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.84, Train f1 (macro): 0.68, Val f1 (macro): 0.35, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.25, Val Loss: 0.47, Train f1 (macro): 0.92, Val f1 (macro): 0.45, Elapsed time: 3m 37s
EPOCH [20/500],	 Train Loss: 0.15, Val Loss: 0.51, Train f1 (macro): 0.96, Val f1 (macro): 0.46, Elapsed time: 7m 14s
Training completed in 7m 58s for GRU.
Total epochs trained: 22
Best epoch: 11
Best val loss: 0.43877929792954373
Best val f1 (macro): 0.45356454358935006
ITERATION 18:
Model: CNNGRU-BTC-ITER18
EPOCH [1/500],	 Train Loss: 0.85, Val Loss: 1.32, Train f1 (macro): 0.56, Val f1 (macro): 0.16, Elapsed time: 0m 23s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.52, Train f1 (macro): 0.91, Val f1 (macro): 0.42

ITERATION 22:
Model: CNNGRU-BTC-ITER22
EPOCH [1/500],	 Train Loss: 0.85, Val Loss: 1.17, Train f1 (macro): 0.56, Val f1 (macro): 0.18, Elapsed time: 0m 23s
EPOCH [10/500],	 Train Loss: 0.20, Val Loss: 0.46, Train f1 (macro): 0.93, Val f1 (macro): 0.44, Elapsed time: 3m 37s
EPOCH [20/500],	 Train Loss: 0.10, Val Loss: 0.34, Train f1 (macro): 0.97, Val f1 (macro): 0.44, Elapsed time: 7m 11s
EPOCH [30/500],	 Train Loss: 0.04, Val Loss: 0.35, Train f1 (macro): 0.99, Val f1 (macro): 0.45, Elapsed time: 10m 53s
Training completed in 11m 60s for CNNGRU.
Total epochs trained: 33
Best epoch: 22
Best val loss: 0.3219656272289845
Best val f1 (macro): 0.4519169738655267
------------------------------------------------------------------
Model: CNN-BTC-ITER22
EPOCH [1/500],	 Train Loss: 0.67, Val Loss: 0.98, Train f1 (macro): 0.68, Val f1 (macro): 0.27, Elapsed time: 0m 17s
EPOCH [10/500],	 Train Loss: 0.24, Val Loss: 0.40, Train f1 (macro): 0.92, Val f1 (macro): 0.46, Elapsed time: 2m 43s
EPOCH [20

EPOCH [1/500],	 Train Loss: 0.84, Val Loss: 1.08, Train f1 (macro): 0.57, Val f1 (macro): 0.24, Elapsed time: 0m 23s
EPOCH [10/500],	 Train Loss: 0.22, Val Loss: 0.32, Train f1 (macro): 0.92, Val f1 (macro): 0.45, Elapsed time: 3m 36s
EPOCH [20/500],	 Train Loss: 0.08, Val Loss: 0.35, Train f1 (macro): 0.97, Val f1 (macro): 0.46, Elapsed time: 7m 10s
Training completed in 7m 10s for CNNGRU.
Total epochs trained: 20
Best epoch: 9
Best val loss: 0.3232443905793704
Best val f1 (macro): 0.45259030248418347
------------------------------------------------------------------
Model: CNN-BTC-ITER26
EPOCH [1/500],	 Train Loss: 0.67, Val Loss: 1.04, Train f1 (macro): 0.68, Val f1 (macro): 0.31, Elapsed time: 0m 16s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.73, Train f1 (macro): 0.92, Val f1 (macro): 0.41, Elapsed time: 2m 41s
EPOCH [20/500],	 Train Loss: 0.20, Val Loss: 0.56, Train f1 (macro): 0.93, Val f1 (macro): 0.43, Elapsed time: 5m 23s
Training completed in 6m 27s for CNN.
Total epochs

### Reporting: BTC dataset

In [12]:
BTC_CNNGRU_test_precision = []
BTC_CNNGRU_test_recall = []
BTC_CNNGRU_test_f1_score = []
BTC_CNNGRU_test_annualised_returns = []
for i in range(NUM_LOOPS):
    BTC_CNNGRU_test_precision.append(BTC_CNNGRU_allstats[f"CNNGRU-BTC-ITER{i}"]["avg_macro"]["test"]["precision"])
    BTC_CNNGRU_test_recall.append(BTC_CNNGRU_allstats[f"CNNGRU-BTC-ITER{i}"]["avg_macro"]["test"]["recall"])
    BTC_CNNGRU_test_f1_score.append(BTC_CNNGRU_allstats[f"CNNGRU-BTC-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    BTC_CNNGRU_test_annualised_returns.append(BTC_CNNGRU_allstats[f"CNNGRU-BTC-ITER{i}"]["annualised_returns"])

In [13]:
BTC_CNN_test_precision = []
BTC_CNN_test_recall = []
BTC_CNN_test_f1_score = []
BTC_CNN_test_annualised_returns = []
for i in range(NUM_LOOPS):
    BTC_CNN_test_precision.append(BTC_CNN_allstats[f"CNN-BTC-ITER{i}"]["avg_macro"]["test"]["precision"])
    BTC_CNN_test_recall.append(BTC_CNN_allstats[f"CNN-BTC-ITER{i}"]["avg_macro"]["test"]["recall"])
    BTC_CNN_test_f1_score.append(BTC_CNN_allstats[f"CNN-BTC-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    BTC_CNN_test_annualised_returns.append(BTC_CNN_allstats[f"CNN-BTC-ITER{i}"]["annualised_returns"])

In [14]:
BTC_GRU_test_precision = []
BTC_GRU_test_recall = []
BTC_GRU_test_f1_score = []
BTC_GRU_test_annualised_returns = []
for i in range(NUM_LOOPS):
    BTC_GRU_test_precision.append(BTC_GRU_allstats[f"GRU-BTC-ITER{i}"]["avg_macro"]["test"]["precision"])
    BTC_GRU_test_recall.append(BTC_GRU_allstats[f"GRU-BTC-ITER{i}"]["avg_macro"]["test"]["recall"])
    BTC_GRU_test_f1_score.append(BTC_GRU_allstats[f"GRU-BTC-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    BTC_GRU_test_annualised_returns.append(BTC_GRU_allstats[f"GRU-BTC-ITER{i}"]["annualised_returns"])

In [15]:
# Writing results to json file
with open('./statistics/BTC_CNNGRU_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(BTC_CNNGRU_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)
with open('./statistics/BTC_CNN_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(BTC_CNN_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)
with open('./statistics/BTC_GRU_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(BTC_GRU_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)

## Training and Testing for the ETH dataset

In [16]:
ETH_CNNGRU_allstats = {}
ETH_CNN_allstats = {}
ETH_GRU_allstats = {}

In [17]:
# 1.2. Training and testing for ETH dataset
for i in range(NUM_LOOPS):
    print(f"ITERATION {i}:")
    
    # ==============================================================================
    # ETH - CNNGRU
    eth_exec_CNNGRU = ExecutionHandler(dataset=eth, model=CNNGRU,
                                       model_id=f"ETH-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(eth)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {eth_exec_CNNGRU.model.__class__.__name__}-{eth_exec_CNNGRU.model.model_id}")
    eth_exec_CNNGRU.run()
    # Append results
    model_name4 = next(iter(eth_exec_CNNGRU.statistics))
    ETH_CNNGRU_allstats[model_name4] = eth_exec_CNNGRU.statistics[model_name4]
    
    print(f"------------------------------------------------------------------")
    
    # ==============================================================================
    # ETH - CNN
    eth_exec_CNN = ExecutionHandler(dataset=eth, model=CNN,
                                       model_id=f"ETH-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(eth)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {eth_exec_CNN.model.__class__.__name__}-{eth_exec_CNN.model.model_id}")
    eth_exec_CNN.run()
    # Append results
    model_name5 = next(iter(eth_exec_CNN.statistics))
    ETH_CNN_allstats[model_name5] = eth_exec_CNN.statistics[model_name5]
    
    print(f"------------------------------------------------------------------")
    
    # ==============================================================================
    # ETH - GRU
    eth_exec_GRU = ExecutionHandler(dataset=eth, model=GRU,
                                       model_id=f"ETH-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(eth)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {eth_exec_GRU.model.__class__.__name__}-{eth_exec_GRU.model.model_id}")
    eth_exec_GRU.run()
    # Append results
    model_name6 = next(iter(eth_exec_GRU.statistics))
    ETH_GRU_allstats[model_name6] = eth_exec_GRU.statistics[model_name6]
    print(f"==============================================================================")

ITERATION 0:
Model: CNNGRU-ETH-ITER0
EPOCH [1/500],	 Train Loss: 0.81, Val Loss: 1.12, Train f1 (macro): 0.59, Val f1 (macro): 0.24, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.32, Val Loss: 0.87, Train f1 (macro): 0.88, Val f1 (macro): 0.33, Elapsed time: 3m 36s
EPOCH [20/500],	 Train Loss: 0.23, Val Loss: 0.69, Train f1 (macro): 0.91, Val f1 (macro): 0.37, Elapsed time: 7m 12s
Training completed in 9m 43s for CNNGRU.
Total epochs trained: 27
Best epoch: 16
Best val loss: 0.6367717201893146
Best val f1 (macro): 0.3745954532402589
------------------------------------------------------------------
Model: CNN-ETH-ITER0
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 1.06, Train f1 (macro): 0.68, Val f1 (macro): 0.30, Elapsed time: 0m 16s
EPOCH [10/500],	 Train Loss: 0.24, Val Loss: 0.59, Train f1 (macro): 0.91, Val f1 (macro): 0.42, Elapsed time: 2m 44s
EPOCH [20/500],	 Train Loss: 0.16, Val Loss: 0.52, Train f1 (macro): 0.94, Val f1 (macro): 0.44, Elapsed time: 5m 28s
Training compl

EPOCH [20/500],	 Train Loss: 0.26, Val Loss: 0.67, Train f1 (macro): 0.90, Val f1 (macro): 0.35, Elapsed time: 6m 58s
EPOCH [30/500],	 Train Loss: 0.22, Val Loss: 0.68, Train f1 (macro): 0.92, Val f1 (macro): 0.36, Elapsed time: 10m 28s
Training completed in 13m 42s for CNNGRU.
Total epochs trained: 39
Best epoch: 28
Best val loss: 0.667043710557314
Best val f1 (macro): 0.3604126539949067
------------------------------------------------------------------
Model: CNN-ETH-ITER4
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.81, Train f1 (macro): 0.69, Val f1 (macro): 0.37, Elapsed time: 0m 16s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.58, Train f1 (macro): 0.92, Val f1 (macro): 0.42, Elapsed time: 2m 36s
EPOCH [20/500],	 Train Loss: 0.13, Val Loss: 0.46, Train f1 (macro): 0.96, Val f1 (macro): 0.45, Elapsed time: 5m 14s
EPOCH [30/500],	 Train Loss: 0.11, Val Loss: 0.48, Train f1 (macro): 0.96, Val f1 (macro): 0.45, Elapsed time: 7m 51s
Training completed in 10m 12s for CNN.
Total epoc

EPOCH [10/500],	 Train Loss: 0.22, Val Loss: 0.68, Train f1 (macro): 0.92, Val f1 (macro): 0.41, Elapsed time: 2m 45s
EPOCH [20/500],	 Train Loss: 0.16, Val Loss: 0.51, Train f1 (macro): 0.95, Val f1 (macro): 0.44, Elapsed time: 5m 30s
Training completed in 6m 19s for CNN.
Total epochs trained: 23
Best epoch: 12
Best val loss: 0.44155643249933535
Best val f1 (macro): 0.4417424460326647
------------------------------------------------------------------
Model: GRU-ETH-ITER8
EPOCH [1/500],	 Train Loss: 0.68, Val Loss: 0.81, Train f1 (macro): 0.67, Val f1 (macro): 0.36, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.25, Val Loss: 0.65, Train f1 (macro): 0.92, Val f1 (macro): 0.41, Elapsed time: 3m 36s
Training completed in 5m 46s for GRU.
Total epochs trained: 16
Best epoch: 5
Best val loss: 0.27186803319133246
Best val f1 (macro): 0.4608778659764566
ITERATION 9:
Model: CNNGRU-ETH-ITER9
EPOCH [1/500],	 Train Loss: 0.82, Val Loss: 1.37, Train f1 (macro): 0.58, Val f1 (macro): 0.14, Ela

ITERATION 13:
Model: CNNGRU-ETH-ITER13
EPOCH [1/500],	 Train Loss: 0.83, Val Loss: 1.08, Train f1 (macro): 0.57, Val f1 (macro): 0.23, Elapsed time: 0m 21s
EPOCH [10/500],	 Train Loss: 0.24, Val Loss: 0.43, Train f1 (macro): 0.91, Val f1 (macro): 0.42, Elapsed time: 3m 37s
EPOCH [20/500],	 Train Loss: 0.11, Val Loss: 0.39, Train f1 (macro): 0.96, Val f1 (macro): 0.43, Elapsed time: 7m 12s
EPOCH [30/500],	 Train Loss: 0.04, Val Loss: 0.35, Train f1 (macro): 0.99, Val f1 (macro): 0.44, Elapsed time: 10m 47s
Training completed in 11m 8s for CNNGRU.
Total epochs trained: 31
Best epoch: 20
Best val loss: 0.282370249239298
Best val f1 (macro): 0.4373619721056958
------------------------------------------------------------------
Model: CNN-ETH-ITER13
EPOCH [1/500],	 Train Loss: 0.64, Val Loss: 0.86, Train f1 (macro): 0.69, Val f1 (macro): 0.34, Elapsed time: 0m 16s
EPOCH [10/500],	 Train Loss: 0.20, Val Loss: 0.66, Train f1 (macro): 0.93, Val f1 (macro): 0.41, Elapsed time: 2m 43s
Training co

------------------------------------------------------------------
Model: CNN-ETH-ITER17
EPOCH [1/500],	 Train Loss: 0.65, Val Loss: 1.25, Train f1 (macro): 0.69, Val f1 (macro): 0.28, Elapsed time: 0m 17s
EPOCH [10/500],	 Train Loss: 0.20, Val Loss: 0.53, Train f1 (macro): 0.93, Val f1 (macro): 0.44, Elapsed time: 2m 53s
Training completed in 5m 28s for CNN.
Total epochs trained: 19
Best epoch: 8
Best val loss: 0.48189299553632736
Best val f1 (macro): 0.43730110553673246
------------------------------------------------------------------
Model: GRU-ETH-ITER17
EPOCH [1/500],	 Train Loss: 0.67, Val Loss: 0.69, Train f1 (macro): 0.67, Val f1 (macro): 0.38, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.22, Val Loss: 0.50, Train f1 (macro): 0.94, Val f1 (macro): 0.44, Elapsed time: 3m 39s
Training completed in 5m 7s for GRU.
Total epochs trained: 14
Best epoch: 3
Best val loss: 0.4747875619393129
Best val f1 (macro): 0.43586917977843287
ITERATION 18:
Model: CNNGRU-ETH-ITER18
EPOCH [1/

------------------------------------------------------------------
Model: GRU-ETH-ITER21
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.79, Train f1 (macro): 0.68, Val f1 (macro): 0.36, Elapsed time: 0m 21s
EPOCH [10/500],	 Train Loss: 0.21, Val Loss: 0.55, Train f1 (macro): 0.94, Val f1 (macro): 0.44, Elapsed time: 3m 34s
Training completed in 5m 0s for GRU.
Total epochs trained: 14
Best epoch: 3
Best val loss: 0.40013602662544984
Best val f1 (macro): 0.4441237319770901
ITERATION 22:
Model: CNNGRU-ETH-ITER22
EPOCH [1/500],	 Train Loss: 0.82, Val Loss: 1.04, Train f1 (macro): 0.57, Val f1 (macro): 0.23, Elapsed time: 0m 21s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.58, Train f1 (macro): 0.92, Val f1 (macro): 0.42, Elapsed time: 3m 38s
EPOCH [20/500],	 Train Loss: 0.08, Val Loss: 0.34, Train f1 (macro): 0.97, Val f1 (macro): 0.45, Elapsed time: 7m 16s
EPOCH [30/500],	 Train Loss: 0.05, Val Loss: 0.34, Train f1 (macro): 0.98, Val f1 (macro): 0.45, Elapsed time: 10m 52s
Training compl

Training completed in 9m 23s for GRU.
Total epochs trained: 26
Best epoch: 15
Best val loss: 0.47144788847519803
Best val f1 (macro): 0.44898455064498194
ITERATION 26:
Model: CNNGRU-ETH-ITER26
EPOCH [1/500],	 Train Loss: 0.81, Val Loss: 1.27, Train f1 (macro): 0.59, Val f1 (macro): 0.19, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.21, Val Loss: 0.48, Train f1 (macro): 0.92, Val f1 (macro): 0.42, Elapsed time: 3m 36s
EPOCH [20/500],	 Train Loss: 0.11, Val Loss: 0.47, Train f1 (macro): 0.96, Val f1 (macro): 0.43, Elapsed time: 7m 12s
Training completed in 9m 22s for CNNGRU.
Total epochs trained: 26
Best epoch: 15
Best val loss: 0.3097709514773809
Best val f1 (macro): 0.4421577645460168
------------------------------------------------------------------
Model: CNN-ETH-ITER26
EPOCH [1/500],	 Train Loss: 0.63, Val Loss: 0.78, Train f1 (macro): 0.70, Val f1 (macro): 0.36, Elapsed time: 0m 17s
EPOCH [10/500],	 Train Loss: 0.24, Val Loss: 0.41, Train f1 (macro): 0.91, Val f1 (macro): 0.

### Reporting: ETH Dataset

In [18]:
ETH_CNNGRU_test_precision = []
ETH_CNNGRU_test_recall = []
ETH_CNNGRU_test_f1_score = []
ETH_CNNGRU_test_annualised_returns = []
for i in range(NUM_LOOPS):
    ETH_CNNGRU_test_precision.append(ETH_CNNGRU_allstats[f"CNNGRU-ETH-ITER{i}"]["avg_macro"]["test"]["precision"])
    ETH_CNNGRU_test_recall.append(ETH_CNNGRU_allstats[f"CNNGRU-ETH-ITER{i}"]["avg_macro"]["test"]["recall"])
    ETH_CNNGRU_test_f1_score.append(ETH_CNNGRU_allstats[f"CNNGRU-ETH-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    ETH_CNNGRU_test_annualised_returns.append(ETH_CNNGRU_allstats[f"CNNGRU-ETH-ITER{i}"]["annualised_returns"])

In [19]:
ETH_CNN_test_precision = []
ETH_CNN_test_recall = []
ETH_CNN_test_f1_score = []
ETH_CNN_test_annualised_returns = []
for i in range(NUM_LOOPS):
    ETH_CNN_test_precision.append(ETH_CNN_allstats[f"CNN-ETH-ITER{i}"]["avg_macro"]["test"]["precision"])
    ETH_CNN_test_recall.append(ETH_CNN_allstats[f"CNN-ETH-ITER{i}"]["avg_macro"]["test"]["recall"])
    ETH_CNN_test_f1_score.append(ETH_CNN_allstats[f"CNN-ETH-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    ETH_CNN_test_annualised_returns.append(ETH_CNN_allstats[f"CNN-ETH-ITER{i}"]["annualised_returns"])

In [20]:
ETH_GRU_test_precision = []
ETH_GRU_test_recall = []
ETH_GRU_test_f1_score = []
ETH_GRU_test_annualised_returns = []
for i in range(NUM_LOOPS):
    ETH_GRU_test_precision.append(ETH_GRU_allstats[f"GRU-ETH-ITER{i}"]["avg_macro"]["test"]["precision"])
    ETH_GRU_test_recall.append(ETH_GRU_allstats[f"GRU-ETH-ITER{i}"]["avg_macro"]["test"]["recall"])
    ETH_GRU_test_f1_score.append(ETH_GRU_allstats[f"GRU-ETH-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    ETH_GRU_test_annualised_returns.append(ETH_GRU_allstats[f"GRU-ETH-ITER{i}"]["annualised_returns"])

In [21]:
# Writing results to json file
with open('./statistics/ETH_CNNGRU_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(ETH_CNNGRU_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)
with open('./statistics/ETH_CNN_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(ETH_CNN_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)
with open('./statistics/ETH_GRU_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(ETH_GRU_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)

## Training and Testing for the BNB dataset

In [22]:
BNB_CNNGRU_allstats = {}
BNB_CNN_allstats = {}
BNB_GRU_allstats = {}

In [23]:
# 1.2. Training and testing for BNB dataset
for i in range(NUM_LOOPS):
    print(f"ITERATION {i}:")
    
    # ==============================================================================
    # BNB - CNNGRU
    bnb_exec_CNNGRU = ExecutionHandler(dataset=bnb, model=CNNGRU,
                                       model_id=f"BNB-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(bnb)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {bnb_exec_CNNGRU.model.__class__.__name__}-{bnb_exec_CNNGRU.model.model_id}")
    bnb_exec_CNNGRU.run()
    # Append results
    model_name7 = next(iter(bnb_exec_CNNGRU.statistics))
    BNB_CNNGRU_allstats[model_name7] = bnb_exec_CNNGRU.statistics[model_name7]
    
    print(f"------------------------------------------------------------------")
    
    # ==============================================================================
    # BNB - CNN
    bnb_exec_CNN = ExecutionHandler(dataset=bnb, model=CNN,
                                       model_id=f"BNB-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(bnb)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {bnb_exec_CNN.model.__class__.__name__}-{bnb_exec_CNN.model.model_id}")
    bnb_exec_CNN.run()
    # Append results
    model_name8 = next(iter(bnb_exec_CNN.statistics))
    BNB_CNN_allstats[model_name8] = bnb_exec_CNN.statistics[model_name8]
    
    print(f"------------------------------------------------------------------")
    
    # ==============================================================================
    # BNB - GRU
    bnb_exec_GRU = ExecutionHandler(dataset=bnb, model=GRU,
                                       model_id=f"BNB-ITER{i}",
                                       device=DEVICE, batch_size=BATCH_SIZE, lr=LR, 
                                       timeperiod=TIMEPERIOD, random_state=RANDOM_SEED,
                                       train_start_idx=i, test_end_idx=len(bnb)-NUM_LOOPS+i,
                                       train_cutoff=TRAIN_CUTOFF+timedelta(days=i),
                                       val_cutoff=VAL_CUTOFF+timedelta(days=i),
                                       checkpoints_path='./checkpoints-report-20240525',
                                       preload_weights_path=None,
                                       transform1=to_tensor, transform2=normalise,
                                       transform3=reshape_tsr)
    print(f"Model: {bnb_exec_GRU.model.__class__.__name__}-{bnb_exec_GRU.model.model_id}")
    bnb_exec_GRU.run()
    # Append results
    model_name9 = next(iter(bnb_exec_GRU.statistics))
    BNB_GRU_allstats[model_name9] = bnb_exec_GRU.statistics[model_name9]
    print(f"==============================================================================")

ITERATION 0:
Model: CNNGRU-BNB-ITER0
EPOCH [1/500],	 Train Loss: 0.82, Val Loss: 1.18, Train f1 (macro): 0.58, Val f1 (macro): 0.22, Elapsed time: 0m 21s
EPOCH [10/500],	 Train Loss: 0.22, Val Loss: 0.72, Train f1 (macro): 0.92, Val f1 (macro): 0.40, Elapsed time: 3m 35s
EPOCH [20/500],	 Train Loss: 0.09, Val Loss: 0.40, Train f1 (macro): 0.97, Val f1 (macro): 0.44, Elapsed time: 7m 18s
EPOCH [30/500],	 Train Loss: 0.06, Val Loss: 0.39, Train f1 (macro): 0.98, Val f1 (macro): 0.43, Elapsed time: 11m 2s
Training completed in 11m 24s for CNNGRU.
Total epochs trained: 31
Best epoch: 20
Best val loss: 0.37985037467800653
Best val f1 (macro): 0.4410824187117477
------------------------------------------------------------------
Model: CNN-BNB-ITER0
EPOCH [1/500],	 Train Loss: 0.63, Val Loss: 1.18, Train f1 (macro): 0.70, Val f1 (macro): 0.27, Elapsed time: 0m 17s
EPOCH [10/500],	 Train Loss: 0.22, Val Loss: 0.71, Train f1 (macro): 0.92, Val f1 (macro): 0.42, Elapsed time: 2m 52s
Training com

EPOCH [10/500],	 Train Loss: 0.20, Val Loss: 0.56, Train f1 (macro): 0.93, Val f1 (macro): 0.43, Elapsed time: 2m 52s
EPOCH [20/500],	 Train Loss: 0.17, Val Loss: 0.52, Train f1 (macro): 0.94, Val f1 (macro): 0.44, Elapsed time: 5m 45s
Training completed in 6m 20s for CNN.
Total epochs trained: 22
Best epoch: 11
Best val loss: 0.4768291855087647
Best val f1 (macro): 0.4478730167075263
------------------------------------------------------------------
Model: GRU-BNB-ITER4
EPOCH [1/500],	 Train Loss: 0.67, Val Loss: 1.04, Train f1 (macro): 0.68, Val f1 (macro): 0.30, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.64, Train f1 (macro): 0.94, Val f1 (macro): 0.43, Elapsed time: 3m 39s
Training completed in 6m 56s for GRU.
Total epochs trained: 19
Best epoch: 8
Best val loss: 0.5046150288902796
Best val f1 (macro): 0.4465312424333203
ITERATION 5:
Model: CNNGRU-BNB-ITER5
EPOCH [1/500],	 Train Loss: 0.82, Val Loss: 1.19, Train f1 (macro): 0.58, Val f1 (macro): 0.19, Elaps

EPOCH [10/500],	 Train Loss: 0.26, Val Loss: 0.62, Train f1 (macro): 0.92, Val f1 (macro): 0.43, Elapsed time: 3m 60s
EPOCH [20/500],	 Train Loss: 0.15, Val Loss: 0.50, Train f1 (macro): 0.96, Val f1 (macro): 0.45, Elapsed time: 8m 2s
Training completed in 9m 38s for GRU.
Total epochs trained: 24
Best epoch: 13
Best val loss: 0.4060891422514732
Best val f1 (macro): 0.45061215789529574
ITERATION 9:
Model: CNNGRU-BNB-ITER9
EPOCH [1/500],	 Train Loss: 0.81, Val Loss: 1.20, Train f1 (macro): 0.59, Val f1 (macro): 0.18, Elapsed time: 0m 24s
EPOCH [10/500],	 Train Loss: 0.21, Val Loss: 0.44, Train f1 (macro): 0.92, Val f1 (macro): 0.44, Elapsed time: 3m 57s
EPOCH [20/500],	 Train Loss: 0.07, Val Loss: 0.35, Train f1 (macro): 0.98, Val f1 (macro): 0.46, Elapsed time: 7m 51s
Training completed in 9m 25s for CNNGRU.
Total epochs trained: 24
Best epoch: 13
Best val loss: 0.32570267354066557
Best val f1 (macro): 0.45066221120060973
-----------------------------------------------------------------

EPOCH [20/500],	 Train Loss: 0.07, Val Loss: 0.39, Train f1 (macro): 0.98, Val f1 (macro): 0.45, Elapsed time: 7m 7s
Training completed in 7m 7s for CNNGRU.
Total epochs trained: 20
Best epoch: 9
Best val loss: 0.36523360730363774
Best val f1 (macro): 0.4430658029281533
------------------------------------------------------------------
Model: CNN-BNB-ITER13
EPOCH [1/500],	 Train Loss: 0.65, Val Loss: 1.39, Train f1 (macro): 0.69, Val f1 (macro): 0.25, Elapsed time: 0m 16s
EPOCH [10/500],	 Train Loss: 0.22, Val Loss: 0.41, Train f1 (macro): 0.92, Val f1 (macro): 0.45, Elapsed time: 2m 38s
EPOCH [20/500],	 Train Loss: 0.13, Val Loss: 0.45, Train f1 (macro): 0.96, Val f1 (macro): 0.46, Elapsed time: 5m 16s
Training completed in 5m 16s for CNN.
Total epochs trained: 20
Best epoch: 9
Best val loss: 0.4062932391579335
Best val f1 (macro): 0.45096609492638234
------------------------------------------------------------------
Model: GRU-BNB-ITER13
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.

EPOCH [10/500],	 Train Loss: 0.23, Val Loss: 0.57, Train f1 (macro): 0.92, Val f1 (macro): 0.43, Elapsed time: 2m 39s
Training completed in 4m 46s for CNN.
Total epochs trained: 18
Best epoch: 7
Best val loss: 0.4162659381444638
Best val f1 (macro): 0.4493594018959695
------------------------------------------------------------------
Model: GRU-BNB-ITER17
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.85, Train f1 (macro): 0.68, Val f1 (macro): 0.34, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.26, Val Loss: 0.46, Train f1 (macro): 0.92, Val f1 (macro): 0.44, Elapsed time: 3m 40s
Training completed in 6m 13s for GRU.
Total epochs trained: 17
Best epoch: 6
Best val loss: 0.4253537574639687
Best val f1 (macro): 0.4489095146259237
ITERATION 18:
Model: CNNGRU-BNB-ITER18
EPOCH [1/500],	 Train Loss: 0.81, Val Loss: 1.05, Train f1 (macro): 0.58, Val f1 (macro): 0.23, Elapsed time: 0m 21s
EPOCH [10/500],	 Train Loss: 0.21, Val Loss: 0.59, Train f1 (macro): 0.93, Val f1 (macro): 0.43, Ela

EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.85, Train f1 (macro): 0.68, Val f1 (macro): 0.35, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.24, Val Loss: 0.69, Train f1 (macro): 0.93, Val f1 (macro): 0.42, Elapsed time: 3m 39s
Training completed in 5m 50s for GRU.
Total epochs trained: 16
Best epoch: 5
Best val loss: 0.37140750942321926
Best val f1 (macro): 0.44148559396921905
ITERATION 22:
Model: CNNGRU-BNB-ITER22
EPOCH [1/500],	 Train Loss: 0.80, Val Loss: 1.00, Train f1 (macro): 0.58, Val f1 (macro): 0.25, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.22, Val Loss: 0.57, Train f1 (macro): 0.92, Val f1 (macro): 0.43, Elapsed time: 3m 35s
EPOCH [20/500],	 Train Loss: 0.07, Val Loss: 0.35, Train f1 (macro): 0.98, Val f1 (macro): 0.46, Elapsed time: 7m 11s
EPOCH [30/500],	 Train Loss: 0.05, Val Loss: 0.39, Train f1 (macro): 0.98, Val f1 (macro): 0.45, Elapsed time: 10m 49s
Training completed in 10m 49s for CNNGRU.
Total epochs trained: 30
Best epoch: 19
Best val loss: 0.35

ITERATION 26:
Model: CNNGRU-BNB-ITER26
EPOCH [1/500],	 Train Loss: 0.83, Val Loss: 1.05, Train f1 (macro): 0.57, Val f1 (macro): 0.25, Elapsed time: 0m 22s
EPOCH [10/500],	 Train Loss: 0.20, Val Loss: 0.66, Train f1 (macro): 0.93, Val f1 (macro): 0.42, Elapsed time: 3m 38s
EPOCH [20/500],	 Train Loss: 0.06, Val Loss: 0.39, Train f1 (macro): 0.98, Val f1 (macro): 0.46, Elapsed time: 7m 14s
Training completed in 10m 28s for CNNGRU.
Total epochs trained: 29
Best epoch: 18
Best val loss: 0.3748750895834886
Best val f1 (macro): 0.45640368797983494
------------------------------------------------------------------
Model: CNN-BNB-ITER26
EPOCH [1/500],	 Train Loss: 0.66, Val Loss: 0.88, Train f1 (macro): 0.68, Val f1 (macro): 0.34, Elapsed time: 0m 16s
EPOCH [10/500],	 Train Loss: 0.24, Val Loss: 0.55, Train f1 (macro): 0.91, Val f1 (macro): 0.43, Elapsed time: 2m 41s
EPOCH [20/500],	 Train Loss: 0.13, Val Loss: 0.44, Train f1 (macro): 0.96, Val f1 (macro): 0.45, Elapsed time: 5m 22s
EPOCH [30

### Reporting: BNB Dataset

In [24]:
BNB_CNNGRU_test_precision = []
BNB_CNNGRU_test_recall = []
BNB_CNNGRU_test_f1_score = []
BNB_CNNGRU_test_annualised_returns = []
for i in range(NUM_LOOPS):
    BNB_CNNGRU_test_precision.append(BNB_CNNGRU_allstats[f"CNNGRU-BNB-ITER{i}"]["avg_macro"]["test"]["precision"])
    BNB_CNNGRU_test_recall.append(BNB_CNNGRU_allstats[f"CNNGRU-BNB-ITER{i}"]["avg_macro"]["test"]["recall"])
    BNB_CNNGRU_test_f1_score.append(BNB_CNNGRU_allstats[f"CNNGRU-BNB-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    BNB_CNNGRU_test_annualised_returns.append(BNB_CNNGRU_allstats[f"CNNGRU-BNB-ITER{i}"]["annualised_returns"])

In [25]:
BNB_CNN_test_precision = []
BNB_CNN_test_recall = []
BNB_CNN_test_f1_score = []
BNB_CNN_test_annualised_returns = []
for i in range(NUM_LOOPS):
    BNB_CNN_test_precision.append(BNB_CNN_allstats[f"CNN-BNB-ITER{i}"]["avg_macro"]["test"]["precision"])
    BNB_CNN_test_recall.append(BNB_CNN_allstats[f"CNN-BNB-ITER{i}"]["avg_macro"]["test"]["recall"])
    BNB_CNN_test_f1_score.append(BNB_CNN_allstats[f"CNN-BNB-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    BNB_CNN_test_annualised_returns.append(BNB_CNN_allstats[f"CNN-BNB-ITER{i}"]["annualised_returns"])

In [26]:
BNB_GRU_test_precision = []
BNB_GRU_test_recall = []
BNB_GRU_test_f1_score = []
BNB_GRU_test_annualised_returns = []
for i in range(NUM_LOOPS):
    BNB_GRU_test_precision.append(BNB_GRU_allstats[f"GRU-BNB-ITER{i}"]["avg_macro"]["test"]["precision"])
    BNB_GRU_test_recall.append(BNB_GRU_allstats[f"GRU-BNB-ITER{i}"]["avg_macro"]["test"]["recall"])
    BNB_GRU_test_f1_score.append(BNB_GRU_allstats[f"GRU-BNB-ITER{i}"]["avg_macro"]["test"]["f1_score"])
    BNB_GRU_test_annualised_returns.append(BNB_GRU_allstats[f"GRU-BNB-ITER{i}"]["annualised_returns"])

In [27]:
# Writing results to json file
with open('./statistics/BNB_CNNGRU_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(BNB_CNNGRU_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)
with open('./statistics/BNB_CNN_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(BNB_CNN_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)
with open('./statistics/BNB_GRU_allstats.json', 'w', encoding='utf-8') as f:
    json.dump(BNB_GRU_allstats, f, ensure_ascii=False, indent=4, cls=NumpyEncoder)